In [ ]:
import os
import base64
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import BytesIO
import sys, os
sys.path.append(os.path.abspath('..'))

# ✅ No change to user_profile import
from user_profile import get_project_paths

notebook_dir = os.path.abspath(os.path.dirname(__file__)) if "__file__" in locals() else os.getcwd()
outputs = os.path.join(notebook_dir, "..", "data", "outputs")
report_dir = os.path.join(notebook_dir, "..", "report")
summary = pd.read_csv(os.path.join(outputs, 'coverage_summary.csv'))

paths = get_project_paths()
template_path = paths['template_path']

# Plot to memory buffer
plt.figure(figsize=(8, 6))
sns.barplot(
    data=summary.melt(id_vars='Group', var_name='Indicator', value_name='Coverage'),
    x='Indicator', y='Coverage', hue='Group'
)
plt.title('Population-Weighted Coverage by Group')
plt.ylabel('% Coverage')
plt.tight_layout()

buf = BytesIO()
plt.savefig(buf, format='png')
encoded_plot = base64.b64encode(buf.getvalue()).decode('utf-8')
buf.close()

# Load template
with open(os.path.join(template_path, "report_template.html"), "r") as f:
    html_template = f.read()

# Inject table and image
final_html = html_template.replace("{{ table }}", summary.to_html(index=False)).replace("{{ plot }}", encoded_plot)

# Save HTML report
report_path = os.path.join(report_dir, 'coverage_report.html')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(final_html)

print(f"✅ HTML report saved at: {report_path}")
